In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./samples/MNIST_data', one_hot=True)

Extracting ./samples/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./samples/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

소프트맥스 회귀 모델 만들기

이 장에서 우리는 단일 선형 레이어로 된 소프트맥스 회귀 모델을 만들 것입니다. 다음 장에서, 우리는 이 모델을 다중 레이어 합성곱 네트워크로 된 소프트맥스 회귀로 확장할 것입니다.



In [3]:
#Placeholders: 입력 이미지와 목표 출력 클래스를 위한 노드를 만들어서 계산 그래프를 만들자
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#Variables

W = tf.Variable(tf.zeros([784,10])) #W는 784*10 행렬
b = tf.Variable(tf.zeros([10])) #b는 10차원 백터

sess.run(tf.initialize_all_variables())#항상 세션으로 사용하기 전에 초기화

예측 클래스 및 비용 함수(Cost Function)

이제 우리 회귀 모델을 만들 수 있습니다. 딱 한 줄만 필요합니다! 벡터화된 입력 이미지 x를 가중치 행렬 W로 곱하고, 편향(bias) b를 더한 다음 각 클래스에 지정된 소프트맥스 확률들을 계산합니다.

In [4]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

#훈련 과정에소 최소화 될 비용 함수들을 지정. 비용함수는 목표와 모델 예측간의
#교차 엔트로피

cross_entropy = -tf.reduce_sum(y * tf.log(y))

모델 훈련시키기

이제 모델과 훈련 비용 함수를 정의했으므로, TensorFlow를 사용하여 모델을 훈련하는 것은 간단합니다. TensorFlow는 전체 계산 그래프를 알고 있으므로, 각 변수들에 대하여 비용 그라디언트를 찾기 위해 자동 미분을 사용할 수 있습니다. TensorFlow에는 다양한 내장 최적화 알고리즘들 이 있습니다. 이 예제에서는 교차 엔트로피를 하강시키기 위해 가파른 그라디언트 하강법 (steepest gradient descent) 을 0.01의 보폭으로 사용할 것입니다.

In [5]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
#이 한 줄에서 TensorFlow가 진짜로 하는 일은 
#계산 그래프에 새 작업을 추가하는 일입니다. 
#이러한 작업들에는 그라디언트를 계산하고, 매개 변수가 
#갱신 보폭을 계산하고, 갱신된 보폭을 매개 변수에 반영하는 단계를 포함하고 있습니다.

for i in range(1000):
    batch = mnist.train.next_batch(50) #50개 훈련 예제 불러오기
    train_step.run(feed_dict = {x: batch[0], y_: batch[1]})

#feed dict를 활용하면 계산 그래프 내의 어떠한 텐서도 대체 가능

모델 평가하기

우리 모델이 얼마나 좋을까요?

첫번째로 우리가 맞는 라벨을 예측했는지를 확인할 것입니다.. tf.argmax는 특정한 축을 따라 가장 큰 원소의 색인을 알려주는 엄청나게 유용한 함수입니다. 예를 들어 tf.argmax(y,1) 는 진짜 라벨이 tf.argmax(y_,1) 일때 우리 모델이 각 입력에 대하여 가장 정확하다고 생각하는 라벨입니다. 우리는 tf.equal 을 이용해 예측이 실제와 맞았는지 확인할 수 있습니다.

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

#결과는 부울 리스트를 주고 부정 소솟점으로 캐스팅한 후 평균값 구하기
#예를 들어, [True, False, True, True]는 [1,0,1,1] 이 되고 평균값은 0.75

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.098


## 다중 레이어 합성곱 네트워크


가중치 초기화

이 모델을 만들기 위해서, 우리는 많은 가중치 및 편향을 만들 필요가 있습니다. 일반적으로 작은 크기의 노이즈로 가중치를 초기화해서 대칭성을 파괴하고, 그라디언트가 0이 되는 경우를 방지합니다. 우리가 ReLU 뉴런을 사용할 것이므로, 그것들을 약간 양수의 편향을 주어 초기화해서 "죽은 뉴런" 이 되는 것을 막는 것도 좋은 시도입니다. 모델을 만들 때 마다 이 과정을 반복하는 대신, 우리를 위해 이걸 해줄 편리한 두 함수를 만듭시다.

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

합성곱과 풀링 (Pooling)

TensorFlow는 또한 합성곱과 풀링 작업을 위한 많은 유연성을 제공합니다. 어떻게 경계를 다룰까요? 이동 (stride) 크기는 얼마인가요? 이 예시에서, 우리는 언제나 가장 기본적인 버전을 선택할 것입니다. 합성곱은 하나의 이동 크기를 사용하고, 0로 패딩되어 결과적으로 출력의 크기가 입력의 크기와 같게 됩니다. 풀링은 2x2 블럭의 평범한 max pooling 입니다. 코드를 깨끗하게 유지하기 위해 이러한 작업들을 함수로 추상화합시다.

In [11]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


첫 합성곱 레이어

이제 첫번째 레이어를 구현할 수 있습니다. 이 레이어는 합성곱과 max pooling 으로 구성될 것입니다. 합성곱은 5x5 조각 각각마다 32개의 특징을 계산할 것입니다. 가중치 텐서는 [5, 5, 1, 32] 형태가 될 것입니다. 앞의 2 차원은 패치 크기이며, 뒷부분은 출력 채널의 숫자입니다. 또한 각 출력 채널에 대하여 구성 요소 편향 벡터가 있을 것입니다.

In [12]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#레이어를 적용하기 위하여 우선 x를 4차원 텐서의 형태로 변환하는데, 두번째와 세번째 차원을 이미지의 폭과 높이에 해당하고, 
#마지막 차원은 색깔 채널의 수에 대응하는 차원입니다.

x_image = tf.reshape(x, [-1,28,28,1])

#그리고 x_image를 가중치 텐서와 합성곱하고, 편향을 더한 후, Relu함수를 적용 한 후, 마지막으로 max pool 합니다.

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

두번째 합성곱 레이어

심층 네트워크를 만들기 위해 이 유형의 여러 레이어를 쌓습니다. 두번째 레이어는 각각 5x5 조각으로 된 64개의 특징들을 갖고 있습니다.

In [18]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Densely Connected Layer

이제 이미지 크기는 7x7로 줄어 들었습니다. 전체 이미지를 처리할 수 있도록 1024개 뉴런이 있는 전부 연결된 레이어를 추가합니다. 풀링 레이어로부터의 텐서를 벡터들의 배치로 형태를 바꾸고, 가중치 행렬을 곱하고, 편향을 더하고, ReLU를 적용합니다.

In [19]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

탈락 (Dropout)

오버피팅을 줄이기 위해 우리는 판독 레이어 전에 탈락을 적용합니다. 뉴런의 출력이 탈락 동안 유지될 확률에 대한 placeholder를 만듭니다. 이건 훈련 도중에는 탈락을 설정하고 테스트 기간동안에는 해제할 수 있게 해 줍니다. TensorFlow의 tf.nn.dropout 작업은 자동으로 뉴런 출력에 마스킹을 할 뿐 아니라 스케일을 조정해 주므로, 탈락은 어떠한 추가적인 스케일링 없이 동작합니다.

In [21]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

판독 레이어

마지막으로, 위에서 소프트맥스 회귀에서 사용했던것과 같은 소프트맥스 레이어를 추가합니다.

In [25]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

모델 훈련 및 평가

이 모델이 얼마나 좋을까요? 훈련 및 평가를 하기 위해 간단한 단일 레이어 소프트맥스 네트워크에서 사용한것과 거의 같은 코드를 사용할 것입니다. 차이점은 다음과 같습니다: 가파른 그라디언트 하강 최적화 모듈을 더 세련된 ADAM 최적화 모듈로 교체할 것이고, keep_prob 인수를 feed_dict에 포함하여 낙오율을 조정할 것입니다. 그리고 훈련 과정중 100번의 반복마다 기록 (logging)을 추가할 것입니다.

In [26]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./samples/MNIST_data', one_hot=True)

import tensorflow as tf

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,W) + b)

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected Layer

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# Train and Evaluate the Model

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting ./samples/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./samples/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/t10k-labels-idx1-ubyte.gz


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fcc5807be50>> ignored


step 0, training accuracy 0.1
step 100, training accuracy 0.8
step 200, training accuracy 0.92
step 300, training accuracy 0.88
step 400, training accuracy 1
step 500, training accuracy 0.94
step 600, training accuracy 1
step 700, training accuracy 0.9
step 800, training accuracy 0.9
step 900, training accuracy 0.98
step 1000, training accuracy 0.98
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.96
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 0.98
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, traini